In [1]:
import firebase_admin
from firebase_admin import credentials, storage
import pandas as pd
import requests

# Fungsi untuk ekstraksi data dari JSON URL
def extract_data_json(json_url):
    response = requests.get(json_url)
    data = response.json()
    df = pd.DataFrame(data)
    return df

# Fungsi untuk transformasi data (mengambil transaksi saham dengan harga di atas 500 untuk simbol GOOGL, AMZN, MSFT, dan AAPL)
def transform_data(df):
    symbols = ['GOOGL', 'AMZN', 'MSFT', 'AAPL']
    df_transformed = df[df['stock_symbol'].isin(symbols) & (df['trade_price'] > 500)]
    return df_transformed

# Fungsi untuk load ke Google Storage
def load_data_to_firebase(df, storage_bucket, filename):
    try:
        # Cek apakah aplikasi Firebase sudah diinisialisasi sebelumnya
        if not firebase_admin._apps:
            # Jika belum, inisialisasi aplikasi Firebase
            cred = credentials.Certificate("accountKey.json")
            firebase_admin.initialize_app(cred, {
                "storageBucket": storage_bucket
            })
        
        bucket = storage.bucket()
        
        blob = bucket.blob(filename)
        df.to_parquet(filename)
        blob.upload_from_filename(filename, content_type='parquet')
        
        print("Load Data Berhasil")
    except Exception as e:
        print("Error:", e)

# Jalankan fungsi-fungsi secara berurutan
if __name__ == "__main__":
    try:
        # Ekstraksi data dari JSON URL
        json_url = "https://gist.githubusercontent.com/nadirbslmh/93b62fdcfa694d4ec07bed9b3c94e401/raw/c07971341361e23fd4f3a880437c4d8e4ebcfafc/stock_trades.json"
        extracted_df = extract_data_json(json_url)
        
        # Transformasi data
        transformed_df = transform_data(extracted_df)
        
        # Load data yang telah ditransformasi ke Google Storage
        storage_bucket = "de-in-cloud-nuri-hidayatuloh.appspot.com"
        filename = "stock_trades.parquet"
        load_data_to_firebase(transformed_df, storage_bucket, filename)
    except Exception as e:
        print("Error:", e)

Load Data Berhasil
